<a name="top"> <h1>02. Neuronal Networks</h1> <a>

<p>Análisis de sentimiento: Tweets<br />
<strong>Trabajo de Fin de Master</strong><br />
<strong>Master Universitario en Ciencia de Datos</strong></p>

<p>&nbsp;</p>

<p style="text-align:right">V&iacute;ctor Viloria V&aacute;zquez (<em>victor.viloria@cunef.edu</em>)</p>


<hr style="border:1px solid gray">

### Estructura

[1. Librerias utilizadas y funciones](#librerias) 

[2. Introducción ](#introduccion) 

   - Objetivo de negocio.

[3. Yelp Dataset ](#yelp) 

   - Información del dataset
   - Características del dataset


[4. Transformación del formato de ficheros](#transformacion) 


[5. Transformación de datos](#datos)

   - Business
       - Carga del fichero
       - Transformación de los datos
       - Exportación de ficheros procesados

<hr style="border:1px solid gray">

# <a name="librerias"> 1. Librerias utilizadas y funciones <a>


Importamos las librerias a utilizar para el preprocesamiento:

In [1]:
# Import libraries.

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
import numpy as np
import string
import pickle

# Import neural network libraries.

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Dropout, Dense
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

from sklearn.metrics import fbeta_score


# <a name="lectura"> 2. Lectura del dataframe <a>


In [2]:
#Import parquet file.

tweets_df = pd.read_parquet('../../data/processed/tweets.parquet')

# Show the head of the dataframe.

tweets_df.head(10)

,text,sentiment,SentimentText_clean
0,id have responded if i were going,0,id responded going
1,sooo sad i will miss you here in san diego,2,sooo sad miss san diego
2,my boss is bullying me,2,boss bullying
3,what interview leave me alone,2,interview leave alone
4,sons of why couldnt they put them on the rel...,2,sons couldnt put releases already bought
5,some shameless plugging for the best rangers...,0,shameless plugging best rangers forum earth
6,2am feedings for the baby are fun when he is a...,1,2am feedings baby fun smiles coos
7,soooo high,0,soooo high
8,both of you,0,
9,journey wow u just became cooler hehe is tha...,1,journey wow u became cooler hehe possible


In [3]:
# Save in tweets the column "SentimentText_clean" and in sentimientos the column "sentiment".   

tweets = tweets_df["SentimentText_clean"]

sentimientos = tweets_df["sentiment"].values

In [4]:
# Dividir los datos en características y etiquetas
X = tweets
y = tweets_df['sentiment'].values
y = tf.keras.utils.to_categorical(y)  # Convertir los valores numéricos a representación categórica (one-hot encoding)

# División de datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenización y secuenciación del texto
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Save the tokenizer.

with open('../../models/tweets_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Ajuste de la longitud de las secuencias (padding)
max_sequence_length = 100  # Longitud máxima de una secuencia de palabras
X_train_seq = tf.keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen=max_sequence_length)
X_test_seq = tf.keras.preprocessing.sequence.pad_sequences(X_test_seq, maxlen=max_sequence_length)

# Creación del modelo de red neuronal
model = models.Sequential()
model.add(layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_length=max_sequence_length))
model.add(layers.LSTM(64))
model.add(layers.Dense(3, activation='softmax'))


# Compilación del modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenamiento del modelo
#model.fit(X_train_seq, y_train, validation_data=(X_test_seq, y_test), epochs=10, batch_size=32)




In [10]:
#model.save("../../models/nn_reviews.h5")

In [54]:
# Model structure

model.summary()

# Plot model structure

tf.keras.utils.plot_model(model, to_file='nn_reviews.png', show_shapes=True, show_layer_names=True)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          2391000   
                                                                 
 lstm (LSTM)                 (None, 64)                42240     
                                                                 
 dense (Dense)               (None, 3)                 195       
                                                                 
Total params: 2,433,435
Trainable params: 2,433,435
Non-trainable params: 0
_________________________________________________________________
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [5]:
# Load the model.

model = tf.keras.models.load_model('../../models/nn_tweets.h5', compile=False)

with open('../../models/tweets_tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Compilar el modelo
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

max_sequence_length = 100

# Prueba con un texto a introducir
text_to_predict = "I love pizza"
text_to_predict_seq = tokenizer.texts_to_sequences([text_to_predict])
text_to_predict_seq = tf.keras.preprocessing.sequence.pad_sequences(text_to_predict_seq, maxlen=max_sequence_length)
prediction = model.predict(text_to_predict_seq)
sentiment_label = np.argmax(prediction)

# Mapeo de la clase de sentimiento predicha a su etiqueta original (0: neutral, 1: positivo, 2: negativo)
sentiment_labels = ['neutral', 'positivo', 'negativo']
predicted_sentiment = sentiment_labels[sentiment_label]

# Print the predicted sentiment.

print(predicted_sentiment)

1/1 [==============================] - 1s 835ms/step
positivo


In [7]:
# predict the test data.

y_pred = model.predict(X_test_seq)

# Calculate the fbeta score.

fbeta_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1), average='macro', beta=2)


172/172 [==============================] - 5s 26ms/step


0.6525390206829367